In [1]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

In [2]:
income = pd.read_csv("Personal Income Summary Personal Income, Population, Per Capita Personal Income.csv", skiprows=(0,1,2,3,6,7,8,9,10,11,12,13))

In [3]:
del income["GeoFips"]
del income["GeoName"]
del income["LineCode"]
del income["Description"]
income = income.T
income.reset_index(inplace = True)
income.rename(columns = {"index":"dtime", 0:"personal_income"}, inplace = True) #in millions of dollars

In [4]:
income.replace({':Q1':'-01', ':Q2':'-04',':Q3':'-07', ':Q4':'-10',}, regex=True, inplace=True)

In [5]:
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   dtime            79 non-null     object 
 1   personal_income  79 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.4+ KB


In [6]:
income

,dtime,personal_income
0,2001-01,112805.3
1,2001-04,114275.1
2,2001-07,114847.8
3,2001-10,115907.8
4,2002-01,116198.9
...,...,...
74,2019-07,220385.2
75,2019-10,222895.6
76,2020-01,222543.4
77,2020-04,245881.1


In [7]:
income['dtime'] = pd.to_datetime(income['dtime']).dt.to_period('M')

In [8]:
income = income.set_index('dtime').resample('M').interpolate()
income

,personal_income
dtime,
2001-01,112805.300000
2001-02,113295.233333
2001-03,113785.166667
2001-04,114275.100000
2001-05,114466.000000
...,...
2020-03,238101.866667
2020-04,245881.100000
2020-05,242887.366667


In [9]:
income.reset_index(inplace = True)
income['dtime'] = income['dtime'].astype('object')
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   dtime            235 non-null    object 
 1   personal_income  235 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.8+ KB


In [10]:
gdp = pd.read_csv("Gross Domestic Product (GDP) summary, quarterly by state.csv", skiprows=(0,1,2,3,6,7,8,9,10,11,12,13))

In [11]:
gdp

,GeoFips,GeoName,LineCode,Description,2005:Q1,2005:Q2,2005:Q3,2005:Q4,2006:Q1,2006:Q2,...,2018:Q2,2018:Q3,2018:Q4,2019:Q1,2019:Q2,2019:Q3,2019:Q4,2020:Q1,2020:Q2,2020:Q3
0,22000,Louisiana,1,Real GDP (millions of chained 2012 dollars),245979.3,248901.1,244426.8,243602.8,244602.0,243884.5,...,234649.8,235046.0,236872.9,238611.5,238443.3,240978.6,241835.4,234288.9,213207.6,229019.5


In [12]:
del gdp["GeoFips"]
del gdp["GeoName"]
del gdp["LineCode"]
del gdp["Description"]
gdp = gdp.T
gdp.reset_index(inplace = True)
gdp.rename(columns = {"index":"dtime", 0:"real_gdp"}, inplace = True) #in millions of chained 2012 dollars

In [13]:
gdp.replace({':Q1':'-01', ':Q2':'-04',':Q3':'-07', ':Q4':'-10',}, regex=True, inplace=True)

In [14]:
gdp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   dtime     63 non-null     object 
 1   real_gdp  63 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.1+ KB


In [15]:
gdp

,dtime,real_gdp
0,2005-01,245979.3
1,2005-04,248901.1
2,2005-07,244426.8
3,2005-10,243602.8
4,2006-01,244602.0
...,...,...
58,2019-07,240978.6
59,2019-10,241835.4
60,2020-01,234288.9
61,2020-04,213207.6


In [16]:
gdp['dtime'] = pd.to_datetime(gdp['dtime']).dt.to_period('M')

In [17]:
gdp = gdp.set_index('dtime').resample('M').interpolate()
gdp

,real_gdp
dtime,
2005-01,245979.300000
2005-02,246953.233333
2005-03,247927.166667
2005-04,248901.100000
2005-05,247409.666667
...,...
2020-03,220234.700000
2020-04,213207.600000
2020-05,218478.233333


In [18]:
gdp.reset_index(inplace = True)
gdp['dtime'] = gdp['dtime'].astype('object')
gdp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   dtime     187 non-null    object 
 1   real_gdp  187 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.0+ KB


In [19]:
consumer_spending = pd.read_csv("Total personal consumption expenditures (PCE).csv", skiprows=(0,1,2,3))

In [20]:
consumer_spending = consumer_spending.iloc[0:1]

In [21]:
del consumer_spending["GeoFips"]
del consumer_spending["GeoName"]
del consumer_spending["LineCode"]
del consumer_spending["Description"]
consumer_spending = consumer_spending.T
consumer_spending.reset_index(inplace = True)
consumer_spending.rename(columns = {"index":"dtime", 0:"consumer_spending"}, inplace = True) #in millions of chained 2012 dollars

In [22]:
consumer_spending.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dtime              19 non-null     object 
 1   consumer_spending  19 non-null     float64
dtypes: float64(1), object(1)
memory usage: 432.0+ bytes


In [23]:
consumer_spending.dtime = consumer_spending.astype('int')
consumer_spending.dtime += 1
consumer_spending.dtime = consumer_spending.astype('str')
consumer_spending

,dtime,consumer_spending
0,2002,90022.5
1,2003,93570.2
2,2004,98760.7
3,2005,104689.4
4,2006,109377.5
5,2007,116018.7
6,2008,122382.6
7,2009,127072.9
8,2010,126669.9
9,2011,131421.1


In [24]:
consumer_spending['dtime'] = pd.to_datetime(consumer_spending['dtime']).dt.to_period('M')
consumer_spending.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype    
---  ------             --------------  -----    
 0   dtime              19 non-null     period[M]
 1   consumer_spending  19 non-null     float64  
dtypes: float64(1), period[M](1)
memory usage: 432.0 bytes


In [25]:
consumer_spending = consumer_spending.set_index('dtime').resample('M').interpolate()
consumer_spending

,consumer_spending
dtime,
2002-01,90022.500000
2002-02,90318.141667
2002-03,90613.783333
2002-04,90909.425000
2002-05,91205.066667
...,...
2019-09,171476.333333
2019-10,171899.000000
2019-11,172321.666667


In [26]:
consumer_spending.reset_index(inplace = True)
consumer_spending['dtime'] = consumer_spending['dtime'].astype('object')
consumer_spending

,dtime,consumer_spending
0,2002-01,90022.500000
1,2002-02,90318.141667
2,2002-03,90613.783333
3,2002-04,90909.425000
4,2002-05,91205.066667
...,...,...
212,2019-09,171476.333333
213,2019-10,171899.000000
214,2019-11,172321.666667
215,2019-12,172744.333333


In [27]:
income_gdp = pd.merge(income, gdp, on="dtime")
income_gdp

,dtime,personal_income,real_gdp
0,2005-01,131337.100000,245979.300000
1,2005-02,131869.200000,246953.233333
2,2005-03,132401.300000,247927.166667
3,2005-04,132933.400000,248901.100000
4,2005-05,134956.333333,247409.666667
...,...,...,...
182,2020-03,238101.866667,220234.700000
183,2020-04,245881.100000,213207.600000
184,2020-05,242887.366667,218478.233333
185,2020-06,239893.633333,223748.866667


In [28]:
economic = pd.merge(income_gdp, consumer_spending, on="dtime")
economic

,dtime,personal_income,real_gdp,consumer_spending
0,2005-01,131337.100000,245979.300000,104689.400000
1,2005-02,131869.200000,246953.233333,105080.075000
2,2005-03,132401.300000,247927.166667,105470.750000
3,2005-04,132933.400000,248901.100000,105861.425000
4,2005-05,134956.333333,247409.666667,106252.100000
...,...,...,...,...
176,2019-09,222058.800000,241549.800000,171476.333333
177,2019-10,222895.600000,241835.400000,171899.000000
178,2019-11,222778.200000,239319.900000,172321.666667
179,2019-12,222660.800000,236804.400000,172744.333333


In [29]:
economic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181 entries, 0 to 180
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dtime              181 non-null    object 
 1   personal_income    181 non-null    float64
 2   real_gdp           181 non-null    float64
 3   consumer_spending  181 non-null    float64
dtypes: float64(3), object(1)
memory usage: 7.1+ KB


In [30]:
economic['dtime'] = economic['dtime'].astype('str')
economic

,dtime,personal_income,real_gdp,consumer_spending
0,2005-01,131337.100000,245979.300000,104689.400000
1,2005-02,131869.200000,246953.233333,105080.075000
2,2005-03,132401.300000,247927.166667,105470.750000
3,2005-04,132933.400000,248901.100000,105861.425000
4,2005-05,134956.333333,247409.666667,106252.100000
...,...,...,...,...
176,2019-09,222058.800000,241549.800000,171476.333333
177,2019-10,222895.600000,241835.400000,171899.000000
178,2019-11,222778.200000,239319.900000,172321.666667
179,2019-12,222660.800000,236804.400000,172744.333333


In [31]:
economic['dtime'] += "-01"
economic['dtime'] = pd.to_datetime(economic['dtime'])
economic

,dtime,personal_income,real_gdp,consumer_spending
0,2005-01-01,131337.100000,245979.300000,104689.400000
1,2005-02-01,131869.200000,246953.233333,105080.075000
2,2005-03-01,132401.300000,247927.166667,105470.750000
3,2005-04-01,132933.400000,248901.100000,105861.425000
4,2005-05-01,134956.333333,247409.666667,106252.100000
...,...,...,...,...
176,2019-09-01,222058.800000,241549.800000,171476.333333
177,2019-10-01,222895.600000,241835.400000,171899.000000
178,2019-11-01,222778.200000,239319.900000,172321.666667
179,2019-12-01,222660.800000,236804.400000,172744.333333


In [34]:
economic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181 entries, 0 to 180
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   dtime              181 non-null    datetime64[ns]
 1   personal_income    181 non-null    float64       
 2   real_gdp           181 non-null    float64       
 3   consumer_spending  181 non-null    float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 7.1 KB


In [35]:
economic.to_pickle('economic_data.pkl')